# Введение в MapReduce модель на Python


In [85]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [86]:
def MAP(_, row:NamedTuple):
    if (row.gender == 'female'):
        yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if (count > 0):
        yield (age, sum/count)
    else:
        yield (age, 0)

Модель элемента данных

In [87]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str

In [88]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [89]:
def RECORDREADER():
    return [(u.id, u) for u in input_collection]

In [90]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [91]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

In [92]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [93]:
def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

In [94]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [95]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [96]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [97]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykeysorted(iterable):
    t = {}
    for (k2, v2) in iterable:
      t[k2] = t.get(k2, []) + [v2]
    print(t.items())
    print(sorted(t.items(), key=lambda x: x[1]))
    return sorted(t.items(), key=lambda x: x[1])

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [98]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [99]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.6456042614074864),
 (1, 1.6456042614074864),
 (2, 1.6456042614074864),
 (3, 1.6456042614074864),
 (4, 1.6456042614074864)]

## Inverted index 

In [100]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [101]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [102]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [103]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('is', 18)]),
 (1, [('a', 2), ('banana', 2), ('it', 18), ('what', 10)])]

## TeraSort

In [104]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.01778555881234578),
   (None, 0.039306310570093594),
   (None, 0.06935548968517402),
   (None, 0.07977071532624014),
   (None, 0.08273086479111746),
   (None, 0.08559262684360114),
   (None, 0.15813057492290938),
   (None, 0.22893793936287776),
   (None, 0.31194263336534833),
   (None, 0.3355474508232368),
   (None, 0.39774940113434576),
   (None, 0.40471350621305135),
   (None, 0.4143912011658817),
   (None, 0.4518120678895289),
   (None, 0.4630234357722779)]),
 (1,
  [(None, 0.6165816777092237),
   (None, 0.6336644297452622),
   (None, 0.7227116710832118),
   (None, 0.7284485033729011),
   (None, 0.7371009374997344),
   (None, 0.7454667694632627),
   (None, 0.7645594784876816),
   (None, 0.8275532841266855),
   (None, 0.8309710750114676),
   (None, 0.8616578041370991),
   (None, 0.9115279857103239),
   (None, 0.9246149050702891),
   (None, 0.9248260544536382),
   (None, 0.9346389861786729),
   (None, 0.9399259488108899)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [105]:
def record_reader(data):
    for record in data:
        yield record

def map_function(number):
    return [("max", number)]

def reduce_function(key, values):
    return max(values)

def map_reduce_max(input_data):
    mapped_data = []
    for record in record_reader(input_data):
        mapped_data.extend(map_function(record))

    reduced_data = {}
    for key, value in mapped_data:
        reduced_data.setdefault(key, []).append(value)

    max_number = reduce_function("max", reduced_data["max"])
    return max_number

input_data = [10, 5, 7, 20, 100, 25]
result = map_reduce_max(input_data)
print("Максимальное число:", result)

Максимальное число: 100


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [106]:
def record_reader(data):
    for record in data:
        yield record

def map_function(number):
    return [("sum_and_count", (number, 1))]

def reduce_function(key, values):
    total_sum = 0
    total_count = 0
    for value in values:
        total_sum += value[0]
        total_count += value[1]
    mean_value = total_sum / total_count if total_count else 0
    return mean_value

def map_reduce_mean(input_data):
    mapped_data = []
    for record in record_reader(input_data):
        mapped_data.extend(map_function(record))

    reduced_data = {}
    for key, value in mapped_data:
        reduced_data.setdefault(key, []).append(value)

    mean_number = reduce_function("sum_and_count", reduced_data["sum_and_count"])
    return mean_number

input_data = [10, 5, 7, 20, 100, 25]

result = map_reduce_mean(input_data)
print("Арифметическое среднее чисел:", result)

Арифметическое среднее чисел: 27.833333333333332


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [107]:
def record_reader(data):
    for record in data:
        yield record

def map_function(record):
    return record

def sort_and_group(mapped_data):
    sorted_data = sorted(mapped_data, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key in grouped_data:
            grouped_data[key].append(value)
        else:
            grouped_data[key] = [value]
    return grouped_data

def map_reduce_group_by_key(input_data):
    mapped_data = []
    for record in record_reader(input_data):
        mapped_data.append(map_function(record))

    grouped_data = sort_and_group(mapped_data)

    return grouped_data

input_data = [("apple", 1), ("banana", 2), ("apple", 3), ("banana", 4), ("apple", 5)]

result = map_reduce_group_by_key(input_data)
result


{'apple': [1, 3, 5], 'banana': [2, 4]}

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [108]:
def map_function(value):
    return [(value, None)]

def reduce_function(key, _):
    return key

def map_reduce_distinct(input_data):
    mapped_data = []
    for value in input_data:
        mapped_data.extend(map_function(value))

    sorted_mapped_data = sorted(mapped_data, key=lambda x: x[0])
    grouped_data = {}
    for key, _ in sorted_mapped_data:
        grouped_data.setdefault(key, None)

    distinct_data = [reduce_function(key, _) for key in grouped_data]
    return distinct_data

input_data = [1, 2, 3, 2, 3, 4, 5, 5, 6, 1]

result = map_reduce_distinct(input_data)
result

[1, 2, 3, 4, 5, 6]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [109]:
def MAP_SELECT(_, row: NamedTuple):
    if row.age == 25:
        yield (row, row)


def REDUCE_SELECT(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT)
output = list(output)
output

[(User(id=1, age=25, social_contacts=240, gender='female'),
  [User(id=1, age=25, social_contacts=240, gender='female')]),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  [User(id=2, age=25, social_contacts=500, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [110]:
input_collection_2 = [
    dict(id=0, age=54, gender="male", social_contacts=20),
    dict(id=0, age=25, gender="female", social_contacts=240),
    dict(id=1, age=27, gender="undefined", social_contacts=642),
    dict(id=2, age=16, gender="male", social_contacts=123),
    dict(id=2, age=35, gender="undefined", social_contacts=247),
    dict(id=3, age=31, gender="male", social_contacts=521),
    dict(id=3, age=32, gender="female", social_contacts=753),
]

def MAP_PROJECTION(_, row: dict):
    S = ["male", "female"]
    if row["gender"] in S:
        yield (row["id"], row)
    else:
        modified_row = row.copy()
        del modified_row["gender"]
        yield (modified_row["id"], modified_row)


def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u["id"], u) for u in input_collection_2]


output = MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION)
output = list(output)
output

[(0,
  [{'id': 0, 'age': 54, 'gender': 'male', 'social_contacts': 20},
   {'id': 0, 'age': 25, 'gender': 'female', 'social_contacts': 240}]),
 (1, [{'id': 1, 'age': 27, 'social_contacts': 642}]),
 (2,
  [{'id': 2, 'age': 16, 'gender': 'male', 'social_contacts': 123},
   {'id': 2, 'age': 35, 'social_contacts': 247}]),
 (3,
  [{'id': 3, 'age': 31, 'gender': 'male', 'social_contacts': 521},
   {'id': 3, 'age': 32, 'gender': 'female', 'social_contacts': 753}])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [111]:
input_collection_2 = [
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=5, age=27, gender="female", social_contacts=642),
    User(id=6, age=16, gender="male", social_contacts=123),
    User(id=7, age=32, gender="female", social_contacts=753),
]

def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)


def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])


def RECORDREADER():
    return [(u.id, u) for u in input_collection + input_collection_2]


output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=5, age=27, social_contacts=642, gender='female'),
  User(id=5, age=27, social_contacts=642, gender='female')),
 (User(id=6, age=16, social_contacts=123, gender='male'),
  User(id=6, age=16, social_contacts=123, gender='male')),
 (User(id=7, age=32, social_contacts=753, gender='female'),
  User(id=7, age=32, social_contacts=753, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [112]:
def MAP_INTERSECTION(_, row: NamedTuple):
    yield (row.id, row)


def REDUCE_INTERSECTION(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows


def RECORDREADER():
    return [(u.id, u) for u in input_collection + input_collection_2]


output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[[User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [113]:
def MAP_DIFFERENCE(collection_id, user):
    yield (user, collection_id)


def REDUCE_DIFFERENCE(user, collections):
    if collections == [0]:
        yield (user)


def RECORDREADER():
    return [(0, a) for a in input_collection] + [(1, b) for b in input_collection_2]


output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output
     

[User(id=0, age=55, social_contacts=20, gender='male'),
 User(id=1, age=25, social_contacts=240, gender='female'),
 User(id=2, age=25, social_contacts=500, gender='female')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [114]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    city_id: int


users_collection = [
    User(id=1, age=54, gender="male", social_contacts=20, city_id=1),
    User(id=2, age=25, gender="female", social_contacts=240, city_id=2),
    User(id=3, age=27, gender="male", social_contacts=642, city_id=2),
    User(id=4, age=16, gender="male", social_contacts=123, city_id=3),
    User(id=5, age=35, gender="female", social_contacts=247, city_id=4),
]


class City(NamedTuple):
    id: int
    name: str


cities_collection = [
    City(id=1, name="Samara"),
    City(id=2, name="Moscow"),
    City(id=3, name="Omsk"),
    City(id=4, name="Tomks"),
    City(id=5, name="Belgorod"),
]
     

def MAP_JOIN(city_id, row):
    yield (city_id, row)


def REDUCE_JOIN(city_id, rows):
    users = []
    city = None

    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            city = row

    for row in rows:
        if type(row) is User:
            yield (row, row.city_id, city)


def RECORDREADER():
    return [(user.city_id, user) for user in users_collection] + [
        (city.id, city) for city in cities_collection
    ]


output = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
output = list(output)
joined_data = output
joined_data

[(User(id=1, age=54, social_contacts=20, gender='male', city_id=1),
  1,
  City(id=1, name='Samara')),
 (User(id=2, age=25, social_contacts=240, gender='female', city_id=2),
  2,
  City(id=2, name='Moscow')),
 (User(id=3, age=27, social_contacts=642, gender='male', city_id=2),
  2,
  City(id=2, name='Moscow')),
 (User(id=4, age=16, social_contacts=123, gender='male', city_id=3),
  3,
  City(id=3, name='Omsk')),
 (User(id=5, age=35, social_contacts=247, gender='female', city_id=4),
  4,
  City(id=4, name='Tomks'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [115]:
def MAP_GROUP(city_id, user, city):
    yield (city_id, user)


def REDUCE_GROUP(city_id, rows):
    yield f"city with id={city_id} has {len(rows)} user(s)"


def RECORDREADER():
    return [(city_id, user, city) for user, city_id, city in joined_data]


output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output

['city with id=1 has 1 user(s)',
 'city with id=2 has 2 user(s)',
 'city with id=3 has 1 user(s)',
 'city with id=4 has 1 user(s)']

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [116]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [122]:
import numpy as np

I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(
    I, J
)  
big_mat = np.random.rand(J, K)


def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[j, k])


def MAP(k1, v1):
    (j, k) = k1
    w = v1

    for i in range(I):
        k2 = (i, k)
        v2 = small_mat[i, j] * w
        yield (k2, v2)


def REDUCE(key, values):
    (i, k) = key

    k3 = (i, k)

    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)

Проверьте своё решение

In [123]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output: 
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [124]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [125]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)


def RECORDREADER():
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))


def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)


def REDUCE(key, values):
    (i, k) = key

    k3 = (i, k)

    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)
     

# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [126]:
maps = 2
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))


def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)


def REDUCE(key, values):
    (i, k) = key

    k3 = (i, k)

    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]

solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]

print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[0.13041557 0.49912744 0.55448143 0.20952794 0.43557943 0.38492774
  0.33556193 0.41389773 0.58399155 0.18255579 0.32681077 0.59868289
  0.50268762 0.34194148 0.60142323 0.50516149 0.39157496 0.3470231
  0.30745145 0.22510304 0.53925491 0.51020228 0.4059078  0.6802774
  0.48412658 0.1846846  0.36122016 0.33455428 0.35166389 0.59642171
  0.44034103 0.47560612 0.6285504  0.5647839  0.45990963 0.50499202
  0.14571593 0.38138294 0.12053604 0.55914082]
 [0.49097311 0.70367605 1.12729502 0.27662535 0.59311878 0.83014855
  0.37410205 0.4732491  1.02521261 0.37480612 0.77969932 1.430759
  0.89492318 0.88742397 1.39151476 1.06152897 1.30280121 0.56651866
  1.03798939 0.90057232 0.85273804 0.65469872 0.73550525 1.31290653
  0.7538611  0.63493051 0.9156614  1.21632723 1.07718544 1.1065423
  0.93581322 0.85347683 1.52329608 1.29029874 0.85413196 0.75360134
  0.47859395 0.47004213 0.20488491 0.83265275]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [127]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) 

  global maps
  split_size =  int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) 

  split_size =  int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true

126 key-value pairs were sent over a network.
aa (0, [((0, 0), 0.34721049707395096), ((0, 1), 0.6011430607897797), ((0, 2), 0.4421471498870758), ((0, 3), 0.4949014133366182), ((0, 4), 0.07954217900957397), ((0, 5), 0.4254351849217115), ((0, 6), 0.27274712470860385), ((0, 7), 0.4328941639347816), ((0, 8), 0.002868761043102614), ((0, 9), 0.2809718618405342), ((0, 10), 0.6391307385987197), ((0, 11), 0.5776320841246269), ((0, 12), 0.5906770486168289), ((0, 13), 0.017926184039378547), ((0, 14), 0.6985103740822748), ((0, 15), 0.6469362303638772), ((0, 16), 0.011667581574541088), ((0, 17), 0.5675922590065343), ((0, 18), 0.4282723822036609), ((0, 19), 0.2905849064898332), ((0, 20), 0.5766831630356294), ((0, 21), 0.6809994580739922), ((0, 22), 0.5464492065934028), ((0, 23), 0.22598916233128796), ((0, 24), 0.5057474623261924), ((0, 25), 0.49636682533350923), ((0, 26), 0.39230203757308546), ((0, 27), 0.06374486708207874), ((0, 28), 0.3169796174949649), ((0, 29), 0.5131073105817926), ((0, 30), 0.4

True